In [112]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [113]:
custom_image_path = '/Users/magicalnulu1800/Documents/input'

In [114]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 56 * 56, 64)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

In [115]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [116]:
dataset = ImageFolder(root=custom_image_path, transform=transform)

In [117]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [118]:
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [119]:
model = SimpleCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [120]:
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=64, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

In [121]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)

    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)

    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = val_corrects.double() / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

Epoch 1/10, Train Loss: 0.6522, Val Loss: 0.6304, Val Accuracy: 0.6596
Epoch 2/10, Train Loss: 0.5450, Val Loss: 0.5248, Val Accuracy: 0.7489
Epoch 3/10, Train Loss: 0.4658, Val Loss: 0.4545, Val Accuracy: 0.7992
Epoch 4/10, Train Loss: 0.3942, Val Loss: 0.4628, Val Accuracy: 0.8017
Epoch 5/10, Train Loss: 0.3387, Val Loss: 0.5297, Val Accuracy: 0.7874
Epoch 6/10, Train Loss: 0.2826, Val Loss: 0.5811, Val Accuracy: 0.7864
Epoch 7/10, Train Loss: 0.2220, Val Loss: 0.6058, Val Accuracy: 0.7869
Epoch 8/10, Train Loss: 0.1790, Val Loss: 0.9096, Val Accuracy: 0.7721
Epoch 9/10, Train Loss: 0.1517, Val Loss: 0.9474, Val Accuracy: 0.7815
Epoch 10/10, Train Loss: 0.1281, Val Loss: 0.9090, Val Accuracy: 0.7864


In [125]:
torch.save(model.state_dict(), 'custom_cnn_model.pth')

In [129]:
from PIL import Image

image = Image.open('/Users/magicalnulu1800/Desktop/pictureexample.png')
image = image.convert("RGB")
image_tensor = transform(image).unsqueeze(0).to(device)

In [130]:
model.eval()

with torch.no_grad():
    outputs = model(image_tensor)
    _, predicted = torch.max(outputs, 1)

class_labels = ['Swapped', 'Real-Human']
prediction = class_labels[predicted.item()]

print(f"The image is classified as: {prediction}")

The image is classified as: Swapped
